In [1]:
# import libraries
import numpy as np
import pandas as pd
import datetime
import math
from tqdm import tqdm

In [2]:
# preprocessing the demographic part of the data
def preprocess_demo(df):
    # standardize the inputs in ETHNICITY
    for i in tqdm(range(len(df))):
        if 'ASIAN' in df.loc[i, 'ETHNICITY']:
            df.loc[i, 'ETHNICITY'] = 'ASIAN'
        elif 'BLACK' in df.loc[i, 'ETHNICITY']:
            df.loc[i, 'ETHNICITY'] = 'BLACK'
        elif 'HISPANIC' in df.loc[i, 'ETHNICITY']:
            df.loc[i, 'ETHNICITY'] = 'HISPANIC'
        elif 'WHITE' in df.loc[i, 'ETHNICITY']:
            df.loc[i, 'ETHNICITY'] = 'WHITE'
        else:
            df.loc[i, 'ETHNICITY'] = 'OTHERS'
    
    assert len(pd.unique(df['GENDER'])) == 2
    assert len(pd.unique(df['ETHNICITY'])) == 5
    
    df = pd.get_dummies(df, columns=['GENDER', 'ETHNICITY']) # one-hot-encoding on GENDER and ETHNICITY
    df.reset_index(drop=True, inplace=True)
    # print(len(df))
    return df

In [3]:
# convert the DataTime in SQL to a Python DataTime object
def convert_datetime(df):
    f = '%Y-%m-%d'+' '+'%H:%M:%S'
    df['CHARTTIME'] = df['CHARTTIME'].apply(lambda x: datetime.datetime.strptime(x, f))
    return df

In [4]:
# preprocessing the clinical part of the data
def preprocess_value(df_value, clinical):
    # set column names for the 24 bins for each clinical feature, and the ICUSTAY_ID for merging
    l = ['ICUSTAY_ID']
    for i in range(24):
        l.append('{} {}-{}hr'.format(clinical, i, i+1))
    # print(l)
    new_df_value_array = np.empty((len(pd.unique(df_value['ICUSTAY_ID'])), 25)) # define the new DataFrame array
    cnt = 0 # number of cases processed
    bins = [[] for i in range(24)] # set 24 bins, representing each hour in the first 24 hours of each case
    current_id = df_value.loc[0,'ICUSTAY_ID'] # initialization
    base_time = df_value.loc[0,'CHARTTIME'] # initialization
    for i in range(len(df_value)):
        icustay_id = df_value.loc[i,'ICUSTAY_ID']
        if current_id != icustay_id:
            # perform the averaging over each hour bin
            for idx,e in enumerate(bins):
                if len(e) > 0:
                    bins[idx] = sum(e) / len(e)
                else:
                    bins[idx] = None
            bins.insert(0, current_id)
            new_df_value_array[cnt,:] = np.array(bins, dtype=np.float64) # add preprocessed data
            cnt += 1
            bins = [[] for i in range(24)]
            base_time = df_value.loc[i,'CHARTTIME'] # update base_time
            current_id = icustay_id # update current_id
        time_diff = math.floor((df_value.loc[i,'CHARTTIME']-base_time).total_seconds() / 3600) # time difference in terms of hours
        assert time_diff < 24 # if this generates an error, it is the problem of the generation of .csv file
        bins[time_diff].append(df_value.loc[i,'VALUENUM']) # add a value into the corresponding bin
        
        # check end of for-loop and perform the averaging over each hour bin
        if i == len(df_value)-1:
            for idx,e in enumerate(bins):
                if len(e) > 0:
                    bins[idx] = sum(e) / len(e)
                else:
                    bins[idx] = None
            bins.insert(0, current_id)
            new_df_value_array[cnt,:] = np.array(bins, dtype=np.float64) # add preprocessed data
            cnt += 1
    assert cnt == len(pd.unique(df_value['ICUSTAY_ID']))
    new_df_value = pd.DataFrame(new_df_value_array, columns=l)
    new_df_value['ICUSTAY_ID'] = new_df_value['ICUSTAY_ID'].astype(new_df['ICUSTAY_ID'].dtypes) # change data type so that it can be merged
    return new_df_value

In [5]:
for s in ['cv', 'mv']:
    df_demo_los = pd.read_csv("../input/tabularraw/data/{}/case-demographic-LOS-{}.csv".format(s,s)) # read .csv file for the demographic feature of a system
    new_df = preprocess_demo(df_demo_los) # demographic preprocessing
    for c in tqdm(['hr', 'meanbp', 'spo2', 'fio2', 'temp', 'rr']):
        df_clinical = pd.read_csv("../input/tabularraw/data/{}/{}-{}.csv".format(s,c,s)) # read .csv file for a clinical feature of a system
        df_clinical = convert_datetime(df_clinical) # DateTime conversion from SQL to Python
        df_clinical = preprocess_value(df_clinical, c) # clinical preprocessing
        new_df = new_df.merge(df_clinical, how='left', on='ICUSTAY_ID') # merge by ICUSTAY_ID
    # put the LOS column to the end of DataFrame
    cols = new_df.columns.tolist()
    cols = cols[0:1] + cols[2:] + cols[1:2]
    new_df = new_df[cols]
    new_df.to_csv('{}-preprocess.csv'.format(s),index=False) # export the preprocessed DataFrame

100%|██████████| 6/6 [02:01<00:00, 20.29s/it]
